In [ ]:
pip install openai datasets pygments rouge_score evaluate

Note: you may need to restart the kernel to use updated packages.


In [10]:
from datasets import load_dataset
from openai import OpenAI
from tqdm import tqdm
import evaluate

In [11]:
train_dataset = load_dataset("Jise/ruletaker", split="train")
test_dataset = load_dataset("Jise/ruletaker", split="test")
ood_dataset = load_dataset("Jise/ruletaker", split="ood_test")
print(train_dataset)

rouge = evaluate.load("rouge")

Dataset({
    features: ['context', 'statement', 'label', 'reasoning', 'depth'],
    num_rows: 1000
})


GPT-4o Zero-shot CoT

In [12]:
prompt = """
    {}
    Assertion: {}
"""

instruction = """
    Based on the facts and rules, give simple reasoning steps citing the rules, and output whether the assertion is true.
    You must output in json format: {"reason": "Because {rule1}, {rule2}, ..., {conclusion}", "answer": 1/0}. where rules are copies from the Rule, and conclusion should
    be either the assertion or its negation, do not add other texts.
"""

client = OpenAI()

In [13]:
test_buffer = []
ood_buffer = []

for n, i in enumerate(tqdm(test_dataset)):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": instruction},
            {
                "role": "user",
                "content": prompt.format(i["context"], i["statement"])
            }
        ]
    )
    test_buffer.append(completion.choices[0].message.content)

for n, i in enumerate(ood_dataset):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": instruction},
            {
                "role": "user",
                "content": prompt.format(i["context"], i["statement"])
            }
        ]
    )
    ood_buffer.append(completion.choices[0].message.content)

100%|██████████| 250/250 [05:39<00:00,  1.36s/it]


In [17]:
acc = 0
count = 0
err_count = 0
mapping = {"False": 0, "True": 1}
test_output_reasons = []
ood_output_reasons = []

for i, (o, t) in enumerate(zip(test_buffer, test_dataset)):
    try:
        output = eval(o.strip("```json").strip("```"))
    except:
        err_count += 1
        print("test", i)
        test_output_reasons.append(o)  # If there is output format error, using the raw output as reason
        continue
    count += 1
    if int(output["answer"]) == mapping[t["label"]]:
        acc += 1
    test_output_reasons.append(output["reason"])

print(f"Test Accuracy: {acc/count}")
print(f"Test Output Error: {err_count}")
print(f"Test Reason Rouge: {rouge.compute(predictions=test_output_reasons, references=[i["reasoning"] for i in test_dataset], use_stemmer=True)}")

acc = 0
count = 0
err_count = 0

for i, (o, t) in enumerate(zip(ood_buffer, ood_dataset)):
    try:    
        output = eval(o.strip("```json").strip("```"))
    except:
        err_count += 1
        print("ood", i)
        ood_output_reasons.append(o)
        continue
    if int(output["answer"]) == mapping[t["label"]]:
        acc += 1
    count += 1
    ood_output_reasons.append(output["reason"])
print(f"Extended Accuracy: {acc/count}")
print(f"Extended Error count: {err_count}")
print(f"Extended Reason Rouge: {rouge.compute(predictions=ood_output_reasons, references=[i["reasoning"] for i in ood_dataset], use_stemmer=True)}")

test 31
Test Accuracy: 0.9236947791164659
Test Output Error: 1
Test Reason Rouge: {'rouge1': np.float64(0.5438863580448494), 'rouge2': np.float64(0.4059851395643506), 'rougeL': np.float64(0.4878923631129546), 'rougeLsum': np.float64(0.48863132506752205)}
ood 301
Extended Accuracy: 0.8823529411764706
Extended Error count: 1
Extended Reason Rouge: {'rouge1': np.float64(0.4663651789394434), 'rouge2': np.float64(0.3261430062903753), 'rougeL': np.float64(0.40504670452431935), 'rougeLsum': np.float64(0.40536305878202417)}


GPT-4o Few-shot CoT

{'context': 'Bob is rough. Bob is cold. Dave is blue. Fred is young. Fred is rough. Fred is red. Fred is kind. Fred is cold. Gary is kind. If someone is red then they are blue. If someone is red and green and kind then they are round. If someone is nice and rough then they are round. If someone is rough and blue then they are young. If someone is big and green and blue then they are young. If someone is red and cold and kind then they are big. If someone is blue and red and kind then they are green.',
 'question': 'Fred is not nice.',
 'label': 'entailment',
 'config': 'depth-3ext-NatLang'}

KeyError: "Column reason not in the dataset. Current columns in the dataset: ['context', 'question', 'label', 'config']"